In [123]:
from copy import deepcopy

BOARD_DIRECTIONS = (
    (1,0),
    (1,-1),
    (0,-1),
    (-1,-1),
    (-1,0),
    (-1,1),
    (0,1),
    (1,1),
)

def is_in_board(board, x, y):
    board_width, board_height = get_board_dims(board)
    return 0 <= x < board_width and 0 <= y < board_height

def is_valid_move(board, player, x, y):
    if not is_in_board(board, x, y) or board[y][x] != 0:
        return None
    other_player = get_other_player(player)
    
    board[y][x] = player # Temporarily place tile at x,y
    tiles_to_flip = []
    global BOARD_DIRECTIONS
    for x_delta, y_delta in BOARD_DIRECTIONS:
        x_curr = x + x_delta
        y_curr = y + y_delta
        if not is_in_board(board, x_curr, y_curr):
            continue
        tile_buffer = []
        while board[y_curr][x_curr] == other_player:
            tile_buffer.append((x_curr, y_curr))
            x_curr += x_delta
            y_curr += y_delta
            if not is_in_board(board, x_curr, y_curr):
                break
        if is_in_board(board, x_curr, y_curr) and board[y_curr][x_curr] == player:
            tiles_to_flip.extend(tile_buffer)
    board[y][x] = 0 # Remove temporary tile at x,y
    if len(tiles_to_flip) == 0:
        return None
    return tiles_to_flip

def get_board_dims(board):
    # returns (cols, rows)
    return (len(board[0]), len(board))

def get_valid_moves(board, player):
    cols, rows = get_board_dims(board)
    moves_dict = {}
    for x in range(cols):
        for y in range(rows):
            if board[y][x] == 0:
                tiles_to_flip = is_valid_move(board, player, x, y)
                if tiles_to_flip != None:
                    moves_dict[(x,y)] = tiles_to_flip
    return moves_dict
    
def flip_tiles_on_board(board, player, tiles_to_flip):
    for x,y in tiles_to_flip:
        board[y][x] = player
        
def get_other_player(player):
    if player == 1:
        return 2
    return 1

def evaluate_board(board, player):
    cols, rows = get_board_dims(board)
    other_player = get_other_player(player)
    num_tiles_for_player = 0
    num_tiles_for_other_player = 0
    for x in range(cols):
        for y in range(rows):
            tile = board[y][x]
            if tile == player:
                num_tiles_for_player += 1
            if tile == other_player:
                num_tiles_for_other_player += 1
    return num_tiles_for_player - num_tiles_for_other_player

class Node(object):
    def __init__(self):
        self.children = []
        self.parent = None
        self.eval_val = None
        self.propagated_val = None
        self.move = None # (x, y, player)
        self.board = None
        
def get_num_of_parent_nodes(node):
    depth = 0
    curr_node = node
    while curr_node.parent != None:
        depth += 1
        curr_node = curr_node.parent
    return depth
    
def get_num_of_child_nodes(node):
    depth = 0
    curr_node = node
    while curr_node.children != []:
        depth += 1
        curr_node = curr_node.children[0]
    return depth

def print_board(board):
    cols, rows = get_board_dims(board)
    for y in range(rows):
        line = '|'
        for x in range(cols):
            line += str(board[y][x]) + '|'
        print(line)
    
def build_tree(node, board, player, depth_limit=None, curr_player=None):
    if depth_limit != None and get_num_of_parent_nodes(node) >= depth_limit:
        return
    if curr_player == None:
        curr_player = player
    moves_dict = get_valid_moves(board, curr_player)
    for x,y in moves_dict.keys():
#         print(x,y,curr_player)
        board_copy = deepcopy(board)
#         print(1,board_copy)
        board_copy[y][x] = curr_player # Place player's tile at x,y
#         print(2,board_copy)
#         print('tiles_to_flip', moves_dict[(x,y)])
        flip_tiles_on_board(board_copy, curr_player, moves_dict[(x,y)])
#         print(3,board_copy)
        
        new_node = Node()
        new_node.parent = node
        new_node.eval_val = evaluate_board(board_copy, player)
        new_node.move = (x, y, curr_player)
#         new_node.board = board_copy # For debugging pu
        node.children.append(new_node)

        build_tree(new_node, board_copy, player, depth_limit, curr_player=get_other_player(curr_player))

In [124]:
test_board = [
    [0,0,0],
    [1,2,0],
    [2,1,0],
]
parent_node = Node()
parent_node.board = test_board
build_tree(parent_node, test_board, player=1, depth_limit=2)

(1, 0, 1)
(2, 0, 2)
(0, 0, 2)
(2, 2, 2)
(2, 1, 1)
(2, 0, 2)
(0, 0, 2)
(2, 2, 2)


In [125]:
print(parent_node.children)
print(parent_node.children[0].move)
print(parent_node.children[0].eval_val)
print(parent_node.children[0].board)
print(parent_node.children[0].children[0].move)
print(parent_node.children[0].children[0].eval_val)
print(parent_node.children[0].children[0].board)


[<__main__.Node object at 0x110231810>, <__main__.Node object at 0x10fe87190>]
(1, 0, 1)
3
[[0, 1, 0], [1, 1, 0], [2, 1, 0]]
(2, 0, 2)
0
[[0, 1, 2], [1, 2, 0], [2, 1, 0]]


In [128]:
test_board = [
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,1,2,0,0,0],
    [0,0,0,2,1,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0],
]
parent_node = Node()
parent_node.board = test_board
build_tree(parent_node, test_board, player=1, depth_limit=100)

(4, 2, 1)
(3, 2, 2)
(2, 5, 1)
(5, 4, 2)
(6, 4, 1)
(5, 5, 2)
(4, 6, 1)
(6, 6, 2)
(4, 5, 1)
(7, 3, 2)
(5, 6, 1)
(3, 6, 2)
(2, 7, 1)
(2, 6, 2)
(4, 7, 1)
(2, 3, 2)
(1, 3, 1)
(1, 2, 2)
(5, 7, 1)
(6, 7, 2)
(7, 6, 1)
(5, 2, 2)
(0, 1, 1)
(1, 4, 2)
(6, 1, 1)
(2, 4, 2)
(7, 7, 1)
(7, 0, 2)
(6, 3, 1)
(1, 1, 2)
(0, 4, 1)
(7, 4, 2)
(3, 1, 1)
(2, 0, 2)
(3, 0, 1)
(2, 1, 2)
(0, 2, 1)
(1, 5, 2)
(1, 6, 1)
(0, 7, 2)
(0, 6, 1)
(0, 5, 2)
(2, 2, 1)
(1, 7, 2)
(0, 0, 1)
(3, 7, 2)
(6, 2, 1)
(5, 3, 2)
(6, 0, 1)
(7, 1, 2)
(7, 2, 1)
(5, 0, 2)
(7, 5, 1)
(5, 1, 2)
(0, 3, 1)
(1, 0, 2)
(4, 0, 1)
(4, 1, 2)
(6, 5, 1)
(3, 5, 2)
(3, 5, 1)
(6, 5, 2)
(6, 5, 2)
(4, 1, 1)
(3, 5, 2)
(3, 5, 1)
(4, 1, 2)
(3, 5, 2)
(4, 1, 1)
(6, 5, 2)
(6, 5, 1)
(4, 1, 2)
(4, 1, 1)
(4, 0, 2)
(6, 5, 1)
(3, 5, 2)
(3, 5, 1)
(6, 5, 2)
(6, 5, 2)
(4, 0, 1)
(3, 5, 2)
(3, 5, 1)
(4, 0, 2)
(3, 5, 2)
(6, 5, 1)
(4, 0, 2)
(4, 0, 1)
(6, 5, 2)
(6, 5, 1)
(4, 0, 2)
(4, 1, 1)
(3, 5, 2)
(3, 5, 1)
(4, 1, 2)
(4, 1, 2)
(4, 0, 1)
(3, 5, 2)
(3, 5, 1)
(4, 0, 2)
(3, 5, 2)


KeyboardInterrupt: 

In [119]:
# print(get_num_of_child_nodes(parent_node))

def what_up(node, depth):
    curr_depth = 0
    curr = node
    while curr_depth != depth:
        curr = curr.children[0]
        curr_depth += 1
    print(curr.move)
    print(curr.eval_val)
    print(curr.board)
    print('=================================')

what_up(parent_node, 0)
what_up(parent_node, 1)
what_up(parent_node, 2)
what_up(parent_node, 3)
what_up(parent_node, 4)
what_up(parent_node, 5)





# print(parent_node.children)
# print(parent_node.children[0].move)
# print(parent_node.children[0].eval_val)
# print(parent_node.children[0].board)
# print(parent_node.children[0].children[0].move)
# print(parent_node.children[0].children[0].eval_val)
# print(parent_node.children[0].children[0].board)

None
None
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 2, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
(4, 2, 1)
3
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
(3, 2, 2)
0
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
(5, 1, 2)
-3
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 2, 2, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 2, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
(4, 5, 2)
-8
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 2, 2, 0, 0, 0], [0, 0, 0, 2, 2, 0, 0, 0],

IndexError: list index out of range

In [94]:
import unittest

class ReversiTests(unittest.TestCase):
    
    def test_get_num_of_parent_nodes(self):
        lol = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 0)
        lol.parent = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 1)
        lol.parent.parent = Node()
        self.assertEqual(get_num_of_parent_nodes(lol), 2)
    
    def test_build_tree(self):
        test_board = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        parent_node = Node()
        build_tree(parent_node, test_board, player=1, depth_limit=2)
        self.assertEqual(parent_node.children[0].eval_val, 3)
        self.assertEqual(parent_node.children[0].move, (1, 0, 1))
        self.assertEqual(parent_node.children[1].move, (2, 1, 1))
        self.assertEqual(parent_node.children[0].children[0].move, (2, 0, 2))
        self.assertEqual(parent_node.children[0].children[0].children, [])
    
    def test_flip_tiles_on_board(self):
        original = [
            [0,1,2],
            [1,1,0],
            [2,1,0],
        ]
        after = [
            [0,1,2],
            [1,2,0],
            [2,1,0],
        ]
        flip_tiles_on_board(original, 2, [(1,1)])
        self.assertEqual(original, after)
    
    def test_get_valid_moves(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(get_valid_moves(test_board, 1), {(4, 2): [(3, 2)], (1, 3): [(2, 3)], (3, 1): [(3, 2)], (2, 4): [(2, 3)]})
        
    def test_is_valid_move(self):
        test_board = [
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,1,2,0],
            [0,0,2,1,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board, 1, 3, 1), [(3,2)])
        self.assertEqual(is_valid_move(test_board, 1, 0, 0), None)
        test_board_2 = [
            [0,0,1,0,1],
            [0,0,2,2,0],
            [0,0,0,0,0],
            [0,0,0,0,0],
            [0,0,0,0,0]
        ]
        self.assertEqual(is_valid_move(test_board_2, 1, 2, 2), [(3,1), (2,1)])
        self.assertEqual(is_valid_move(test_board_2, 1, 3, 0), None)
        test_board_3 = [
            [0,0,0],
            [1,2,0],
            [2,1,0],
        ]
        self.assertEqual(is_valid_move(test_board_3, 1, 1, 0), [(1,1)])

    def test_is_in_board(self):
        test_board = [[0]*5]*5
        self.assertEqual(is_in_board(test_board, 4, 4), True)
        self.assertEqual(is_in_board(test_board, 6, 4), False)
        self.assertEqual(is_in_board(test_board, 4, -1), False)

suite = unittest.TestLoader().loadTestsFromTestCase(ReversiTests)
unittest.TextTestRunner(verbosity=2).run(suite)

test_build_tree (__main__.ReversiTests) ... ok
test_flip_tiles_on_board (__main__.ReversiTests) ... ok
test_get_num_of_parent_nodes (__main__.ReversiTests) ... ok
test_get_valid_moves (__main__.ReversiTests) ... ok
test_is_in_board (__main__.ReversiTests) ... ok
test_is_valid_move (__main__.ReversiTests) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [95]:
lol = [[2], [3]]
lmao = list(lol)

In [96]:
lmao[0][0] = 69

In [122]:
get_other_player(2)

1

In [19]:
lol = 'l'
lol += 'adsfg'
print(lol)

ladsfg


In [83]:
original = [
    [0,1,2],
    [1,1,0],
    [2,1,0],
]
after = [
    [0,1,2],
    [1,2,0],
    [2,1,0],
]
flip_tiles_on_board(original, 2, [(1,1)])

In [81]:
original

[[0, 1, 2], [1, 2, 0], [2, 1, 0]]